![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?


In [120]:
# We've loaded your first package for you! You can add as many cells as you need.
import numpy as np
import pandas as pd
import datetime as dt

In [95]:
# importing the data from the files
data_price = pd.read_csv('data/airbnb_price.csv')
data_room_type = pd.ExcelFile('data/airbnb_room_type.xlsx')
data_last_review = pd.read_table('data/airbnb_last_review.tsv')

# parse the first sheet
data_room_type = data_room_type.parse(0)

In [86]:
data_price.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [96]:
# cleaning the price column
data_price['price'] = data_price['price'].str.replace(' dollars','')
data_price['price'] = pd.to_numeric(data_price['price'])

In [124]:
# cleaning price colomn
# Remove outliers
free_listings = data_price['price'] == 0
data_price = data_price.loc[~free_listings]

# Calculate average price
avg_price = round(data_price['price'].mean(), 2)
avg_price


141.82

In [115]:
# calculating the price per month and the avg price
data_price['price_per_month'] = data_price['price'] * 365 / 12

# calculate avg_price_month
avg_price_month = round(data_price['price_per_month'].mean(),2)


# calculating the difference between average cost of an Airbnb listing and the private market
difference = round((avg_price_month - 3100), 2)
avg_price_month

4313.61

In [116]:
# cleaning room_type column
data_room_type['room_type'] = data_room_type['room_type'].str.lower()

# Update the room_type column to category data type
data_room_type['room_type'] = data_room_type['room_type'].astype('category')

# count of room types
room_frequencies = data_room_type['room_type'].value_counts()
room_frequencies

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64

In [121]:
# formating the data type column
data_last_review['last_review'] = pd.to_datetime(data_last_review['last_review'])
first_reviewed = data_last_review['last_review'].dt.date.min()
last_reviewed = data_last_review['last_review'].dt.date.max()

In [104]:
# joining the dataframes prices with room types
rooms_and_prices = data_price.merge(data_room_type, how="outer", on='listing_id')

# joining the dataframes rooms_and_prices with reviews
airbnb_merged = rooms_and_prices.merge(data_last_review, how="outer", on='listing_id')

# droping the null values
airbnb_merged.dropna(inplace=True)
count_duplicate_values = airbnb_merged.duplicated().sum()

# analyzing
airbnb_merged['nbhood_full'].head()
airbnb_merged['borough'] = airbnb_merged['nbhood_full'].str.partition(',')[0]
boroughs = airbnb_merged.groupby('borough')['price'].agg(['sum', 'mean', 'median', 'count'])
boroughs = boroughs.round(2).sort_values("mean", ascending=False)
boroughs.head()

,sum,mean,median,count
borough,,,,
Manhattan,1898417.0,184.04,149.0,10315
Brooklyn,1275250.0,122.02,95.0,10451
Queens,320715.0,92.83,70.0,3455
Staten Island,22974.0,86.04,71.0,267
Bronx,55156.0,79.25,65.0,696


In [118]:
# creating the range price colunm
label_names = ["Budget", "Average", "Expensive", "Extravagant"]
ranges = [0, 69, 175, 350, np.inf]
airbnb_merged['price_range'] = pd.cut(airbnb_merged['price'], bins=ranges, labels=label_names)

# grouping by the columns
prices_borough = airbnb_merged.groupby(['borough', 'price_range'])['price_range'].count()

# showing the results
prices_borough.head()

borough   price_range
Bronx     Budget          381
          Average         285
          Expensive        25
          Extravagant       5
Brooklyn  Budget         3194
Name: price_range, dtype: int64

In [122]:
# creating the dictionary with the finals results
airbnb_analysis = {
    'avg_price': avg_price,
    'average_price_per_month': avg_price_month,
    'difference': difference,
    'room_frequencies': room_frequencies,
    'first_reviewed': first_reviewed,
    'last_reviewed': last_reviewed,
    'prices_by_borough': prices_borough
}

print(airbnb_analysis)

{'avg_price': 92.35, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive   